# Laboratorio de Control Térmico - TempLABUdeA
## Modelado del Sistema - Fase 1

In [1]:

# Parámetros identificados
Kp = 0.3593          # Ganancia del proceso
tm = 10.3349         # Tiempo muerto (s)
tau = 171.7428       # Constante de tiempo (s)

print(f"Modelo FOPDT identificado:")
print(f"G(s) = {Kp} * exp(-{tm} * s) / ({tau} * s + 1)")

Modelo FOPDT identificado:
G(s) = 0.3593 * exp(-10.3349 * s) / (171.7428 * s + 1)


## Sintonía P-only (ITAE)

In [2]:
import sympy as sp

s = sp.symbols('s')
Kp_val = 0.3593
tm_val = 10.3349
tau_val = 171.7428

# Modelo simbólico
G_s = Kp_val * sp.exp(-tm_val * s) / (tau_val * s + 1)
G_s.simplify()

0.3593*exp(-10.3349*s)/(171.7428*s + 1)


### Interpretación:
El modelo FOPDT describe el comportamiento dinámico de la temperatura frente a cambios en la potencia de calefacción con:
- Una **ganancia de proceso** $ K_p = 0.3593 $
- Un **tiempo muerto** $ \theta_p = 10.33\,s $
- Una **constante de tiempo** $\tau_p = 171.74\,s $

Este modelo será utilizado para diseñar el controlador PID en la siguiente fase.
## Diseño de PID mediante Ziegler-Nichols - Fase 2
- Encontrando parametros del PID
- $ K_C =(1.2 * \tau_p) / (K_p * \theta_p) $   
- $ Ti = 2 * \theta_p $
- $ Td = 0.5 * \theta_p $

In [6]:
Kc = (1.2 * tau_val) / (Kp_val * tm)
Ti = 2 * tm
Td = 0.5 * tm
print("Parámetros del controlador PID calculados:")
print(f"Kc = {Kc:.4f}")
print(f"Ti = {Ti:.4f} s")
print(f"Td = {Td:.4f} s")


Parámetros del controlador PID calculados:
Kc = 55.5004
Ti = 20.6698 s
Td = 5.1674 s


### Implementacion  de simulacion

![Simulacio.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Simulacio.png)
![Resultados.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Resultados.png)

- Periodo $P_u = 37.859s$
- $K_p= 104$

- $K_p =0,6K_u$
- $T_i = 0,5P_u$,
- $T_d = 0,125P_u$

In [8]:
Kp=104
Pu=37.859
Ku=Kp*0.6
Ti=0.5*Pu
Td=0.125*Pu
print("Parámetros del controlador PID calculados:") 
print(f"Kc = {Ku:.4f}")
print(f"Ti = {Ti:.4f} s")
print(f"Td = {Td:.4f} s")


Parámetros del controlador PID calculados:
Kc = 62.4000
Ti = 18.9295 s
Td = 4.7324 s


### Implementacion del sistema

$$ 
P=K_c

 $$
  $$
  I=K_c/T_i

 $$
  $$
  D=K_c*T_d
 $$

In [9]:
P=Kc
I=Kc/Ti
D=Kc*Td
print("Parámetros del controlador PID calculados:") 
print(f"Kp = {P:.4f}")
print(f"Ki = {I:.4f}")
print(f"Kd = {D:.4f}")

Parámetros del controlador PID calculados:
Kp = 55.5004
Ki = 2.9320
Kd = 262.6488


![Sistema.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Sistema.png)

### Resultado sistema estabilizado
![Resultado.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Resultado.png)

## Etapa 3 Implementeacion Practica

### Script Python

In [ ]:
"""
Prueba de escalón con registro de datos en Python
Adaptado y documentado para el Laboratorio de Control de Temperatura (TempLabUdeA)
Universidad de Antioquia - Sistemas de Control
"""
! pip install tclab numpy matplotlib pandas datetime
import tclab 
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


# ------------------------------------------------------------------------------
lab = tclab.TCLab()
# Parámetros de control
Kc = 62.4
tauI = 18.9295
tauD = 4.7324
Q_bias = 0.0
ierr = 0.0
prev_err = 0.0

# Inicializar listas (suponiendo que n ya está definido)
n=600
T1 = [0.0] * n
T2 = [0.0] * n
Q1 = [0.0] * n
SP1 = [60.0] * n  # Por ejemplo, setpoint constante de 50

plt.ion()  # Activar modo interactivo
try:
    for i in range(n):
    # Leer temperatura actual
        T1[i] = lab.T1
        T2[i] = lab.T2

        # Cálculo del error
        err = SP1[i] - T1[i]

        # Término integral
        ierr += err

        # Término derivativo
        deriv = err - prev_err
        prev_err = err

        # Acción de control PID
        Q1[i] = Q_bias + Kc * err + (Kc / tauI) * ierr + Kc * tauD * deriv

        # Anti-windup
        if Q1[i] >= 100:
            print(Q1[i])
            Q1[i] = 100
            ierr -= err
        elif Q1[i] <= 0:
            Q1[i] = 0
            ierr -= err

        # Aplicar señal de control
        if (i>10):
            lab.Q1(Q1[i])
        if (i<10):
            Q1=[0.0] * n
        # Gráfica de temperatura
        plt.clf()
        plt.subplot(2, 1, 1)
        plt.plot(T1[:i+1], 'r-o', label='T1')
        plt.plot(T2[:i+1], 'b-o', label='T2')
        plt.ylabel('Temperatura (°C)')
        plt.title('Temperatura')
        plt.grid(True)
        plt.legend()

        # Gráfica de PWM (Q1)
        plt.subplot(2, 1, 2)
        plt.plot(Q1[:i+1], 'b-', label='Q1 (PWM)')
        plt.ylabel('PWM (%)')
        plt.title('Señal de control Q1')
        plt.xlabel('Tiempo (s)')
        plt.grid(True)
        plt.legend()

        plt.pause(0.05)  # Pequeña pausa para actualizar la gráfica
        time.sleep(1)
except KeyboardInterrupt:
    # En caso de interrupción, apagar y cerrar conexión
    print("\nDetención por usuario. Apagando calentadores y cerrando conexión.")
    lab.Q1(0)
    lab.Q2(0)
    lab.close()
    plt.savefig('resultado_prueba.png')
    
plt.show()
t = np.arange(n)

# Crear DataFrame
df = pd.DataFrame({
    'Tiempo (s)': t,
    'Setpoint (SP)': SP1,
    'Temperatura (T)': T1,
    'Control PWM (Q)': Q1
})

# Generar timestamp para los archivos
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Guardar datos en CSV
csv_filename = f'registro_PID_TempLab_{timestamp}.csv'
df.to_csv(csv_filename, index=False)
print(f"Datos guardados en '{csv_filename}'")

# Volver a graficar la última figura para guardarla como imagen
plt.figure(figsize=(10,6))

# Temperatura
plt.subplot(2, 1, 1)
plt.plot(t, T1, 'r-o', label='T1')
plt.plot(t, SP1, 'k--', label='SP')
plt.ylabel('Temperatura (°C)')
plt.title('Temperatura T1 vs Setpoint')
plt.grid(True)
plt.legend()

# PWM
plt.subplot(2, 1, 2)
plt.plot(t, Q1, 'b-', label='Q1 (PWM)')
plt.ylabel('PWM (%)')
plt.xlabel('Tiempo (s)')
plt.title('Señal de control Q1')
plt.grid(True)
plt.legend()

# Guardar imagen
image_filename = f'grafico_PID_TempLab_{timestamp}.png'
plt.tight_layout()
plt.savefig(image_filename)
print(f"Gráfica guardada en '{image_filename}'")

plt.show()


### Respuesta ITAE
![image.png](https://github.com/jbcgames/Lab_5_Control/blob/main/Resultados%20Q1/grafico_PID_TempLab_2025-05-15_17-34-50.png?raw=true)

### Respuesta Zigler Nichols
![image.png](https://github.com/jbcgames/Lab_5_Control/blob/main/Resultados%20Q2/grafico_PID_TempLab_2025-05-15_16-44-19.png?raw=true)

## Etapa 4 Validacion y Analisis

### Script de analisis de datos:


In [16]:
import pandas as pd

def analizar_respuesta_sistema(nombre_archivo_csv, tolerancia=0.05):
    # Leer el archivo
    df = pd.read_csv(nombre_archivo_csv)
    
    tiempo = df["Tiempo (s)"]
    sp = df["Setpoint (SP)"]
    temp = df["Temperatura (T)"]
    
    setpoint = sp.iloc[0]  # asumimos que es constante

    # Calcular sobreimpulso
    temp_max = temp.max()
    overshoot = max(0, ((temp_max - setpoint) / setpoint) * 100)  # en %

    # Calcular error de estado estacionario (últimos 50 valores)
    e_ss = abs(setpoint - temp.tail(50).mean())

    # Calcular tiempo de asentamiento (dentro del ±5% del setpoint)
    margen_superior = setpoint * (1 + tolerancia)
    margen_inferior = setpoint * (1 - tolerancia)

    dentro_margen = (temp >= margen_inferior) & (temp <= margen_superior)
    for i in range(len(temp) - 1, -1, -1):
        if not dentro_margen.iloc[i]:
            settling_time = tiempo.iloc[i + 1] if (i + 1) < len(tiempo) else tiempo.iloc[-1]
            break
    else:
        settling_time = tiempo.iloc[0]  # si nunca sale de la banda

    # Resultados
    
    print(f"Sobreimpulso (%): {overshoot:.2f}")
    print(f"Tiempo de asentamiento (s): {settling_time:.2f}")
    print(f"Error en estado estacionario (°C): {e_ss:.2f}")
    
print("Resultados de la respuesta del sistema con ITAE:")
analizar_respuesta_sistema('https://raw.githubusercontent.com/jbcgames/Lab_5_Control/main/Resultados%20Q1/registro_PID_TempLab_2025-05-15_17-34-50.csv')
print(" ")
print("Resultados de la respuesta del sistema con Ziegler-Nichols:")
analizar_respuesta_sistema('https://raw.githubusercontent.com/jbcgames/Lab_5_Control/main/Resultados%20Q2/registro_PID_TempLab_2025-05-15_16-44-19.csv')

Resultados de la respuesta del sistema con ITAE:
Sobreimpulso (%): 2.07
Tiempo de asentamiento (s): 153.00
Error en estado estacionario (°C): 0.31
 
Resultados de la respuesta del sistema con Ziegler-Nichols:
Sobreimpulso (%): 1.58
Tiempo de asentamiento (s): 171.00
Error en estado estacionario (°C): 0.06


### Comparacion Simulacion practica
- ITAE
